Notebook to analyze the internal air temperature values regarding the ambient temperature

In [3]:
# Python standard library imports
import time

# Third-party imports for database connection and data manipulation
from sqlalchemy import create_engine
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import pearsonr
# Third-party imports for mapping
import folium

In [5]:
# Database connection parameters
dbname = 'ar41'
user = 'postgres'
password = '1234'
host = 'localhost'  # localhost or the server address
port = '5432'  # default PostgreSQL port is 5432

# Establish a connection to the database
connection_str = f"postgresql://{user}:{password}@{host}:{port}/{dbname}"
engine = create_engine(connection_str)

1. Choosing sensors to analyze

    - Out of all the internal sensors, both the Internal Air Temperature PC1 and PC2 are the most correlated to the ambient temperature obtained during the enrichment phase of the project.



2. Logical thinking and procedure:

    1. Null values or values above the acceptable boundaries (65°C) are not considered for this analysis
    2. We first perform a frequency analysis
    3. Using cumulative frequency analysis of the absolute difference between the sensors and the temperature outside of the train we choose the outlier


3. Bucket analysis

In [7]:
query_temperature_differences = f"""
WITH TemperatureDifferences AS (
    SELECT
        mapped_veh_id,
        "timestamps_UTC",
        "Temperature",
        "RS_E_InAirTemp_PC1",
        ABS("Temperature" - "RS_E_InAirTemp_PC1") AS temp_difference
    FROM
        vehicle_data_enriched
    WHERE
        "Temperature" IS NOT NULL
        AND "RS_E_InAirTemp_PC1" < 70
)
SELECT
    bucket_range,
    COUNT(*) AS occurrences
FROM (
    SELECT
        mapped_veh_id,
        "timestamps_UTC",
        "Temperature",
        "RS_E_InAirTemp_PC1",
        temp_difference,
        floor(temp_difference / 5) * 5 AS bucket_range
    FROM
        TemperatureDifferences
) AS temp_diff_buckets
GROUP BY
    bucket_range
ORDER BY
    bucket_range;
"""

In [15]:
df_temperature_differences = pd.read_sql_query(query_temperature_differences, engine)
print(df_temperature_differences)

    bucket_range  occurrences
0            0.0      1240241
1            5.0      2584300
2           10.0      3144885
3           15.0      3789691
4           20.0      3177038
5           25.0      1862560
6           30.0      1024313
7           35.0       492889
8           40.0       203760
9           45.0        87887
10          50.0        40473
11          55.0        13404
12          60.0         3127
13          65.0          409


After here, there are two paths:

- Arbitrarily choosing a threshold or value to consider anomaly (field knowledge)
- Using cumulative frequency analysis to consider only values above a cumulative percentage (e.g. 99%)

In [31]:
# Calculate the cumulative sum of frequencies
df_temperature_differences['Cumulative_Frequency'] = df_temperature_differences['occurrences'].cumsum()

# Calculate the total number of occurrences
total_occurrences = df_temperature_differences['occurrences'].sum()

# Set a threshold percentage
threshold_percentage = 99.9

# Find the bucket where the cumulative frequency exceeds the threshold percentage
outlier_bucket = df_temperature_differences[df_temperature_differences['Cumulative_Frequency'] >= (threshold_percentage / 100 * total_occurrences)]['bucket_range'].min()

print(f"The outlier bucket is {outlier_bucket}") 

The outlier bucket is 50.0


With this analysis, we can say that any timestamp with a temperature difference above 50 shouldn't be normal.

And we can proceed to create our final queries to detect anomalies. For PC1 sensor:

In [32]:
# Analysis of PC1 in air temperature vs outside temperature
query_temperature_differences_PC1 = f"""
SELECT mapped_veh_id, "timestamps_UTC", "Temperature", "RS_E_InAirTemp_PC1", ABS("Temperature"-"RS_E_InAirTemp_PC1")
FROM vehicle_data_enriched
WHERE "Temperature" is not null AND ABS("Temperature"-"RS_E_InAirTemp_PC1") >= 50 AND "RS_E_InAirTemp_PC1" <= 65
GROUP BY mapped_veh_id, "timestamps_UTC", "Temperature", "RS_E_InAirTemp_PC1", ABS("Temperature"-"RS_E_InAirTemp_PC1")
ORDER BY ABS("Temperature"-"RS_E_InAirTemp_PC1") DESC;
"""

In [33]:
df_temperature_differences_PC1 = pd.read_sql_query(query_temperature_differences_PC1, engine)
print(df_temperature_differences_PC1)

       mapped_veh_id       timestamps_UTC  Temperature  RS_E_InAirTemp_PC1  \
0              154.0  2023-02-07 22:50:37         -2.7                65.0   
1              154.0  2023-02-07 22:51:37         -2.7                65.0   
2              154.0  2023-02-07 22:51:57         -2.7                65.0   
3              154.0  2023-02-07 22:52:37         -2.7                65.0   
4              154.0  2023-02-07 22:53:37         -2.7                65.0   
...              ...                  ...          ...                 ...   
48612          197.0  2023-04-03 06:14:54          0.0                50.0   
48613          197.0  2023-04-03 06:19:35          0.0                50.0   
48614          197.0  2023-04-03 06:19:36          0.0                50.0   
48615          197.0  2023-08-07 20:38:07         14.0                64.0   
48616          197.0  2023-08-07 20:38:10         14.0                64.0   

        abs  
0      67.7  
1      67.7  
2      67.7  
3      

And for PC2 sensor:

In [34]:
# Analysis of PC1 in air temperature vs outside temperature
query_temperature_differences_PC2 = f"""
SELECT mapped_veh_id, "timestamps_UTC", "Temperature", "RS_E_InAirTemp_PC2", ABS("Temperature"-"RS_E_InAirTemp_PC2")
FROM vehicle_data_enriched
WHERE "Temperature" is not null AND ABS("Temperature"-"RS_E_InAirTemp_PC2") >= 50 AND "RS_E_InAirTemp_PC2" <= 65
GROUP BY mapped_veh_id, "timestamps_UTC", "Temperature", "RS_E_InAirTemp_PC2", ABS("Temperature"-"RS_E_InAirTemp_PC2")
ORDER BY ABS("Temperature"-"RS_E_InAirTemp_PC2") DESC;
"""

In [35]:
df_temperature_differences_PC2 = pd.read_sql_query(query_temperature_differences_PC2, engine)
print(df_temperature_differences_PC2)

       mapped_veh_id       timestamps_UTC  Temperature  RS_E_InAirTemp_PC2  \
0              126.0  2023-03-02 06:57:38         -1.6                65.0   
1              126.0  2023-03-02 07:12:18         -1.5                65.0   
2              126.0  2023-03-02 07:13:21         -1.5                65.0   
3              126.0  2023-03-02 07:14:21         -1.5                65.0   
4              126.0  2023-03-02 07:14:32         -1.5                65.0   
...              ...                  ...          ...                 ...   
52354          197.0  2023-02-23 10:44:50          8.0                58.0   
52355          197.0  2023-02-23 10:44:54          8.0                58.0   
52356          197.0  2023-02-23 10:45:50          8.0                58.0   
52357          197.0  2023-03-01 21:51:23         -1.0                49.0   
52358          197.0  2023-03-01 21:51:26         -1.0                49.0   

        abs  
0      66.6  
1      66.5  
2      66.5  
3      

With this we have a total of ~100K outliers:

    - Values with absolute difference from outside temperature PC1: 48617
    - Values with absolute difference from outside temperature PC2: 52359